DS5559 Final Project Models

Team Left Twix Members

* Alice Wright - aew7j
* Edward Thompson - ejt8b
* Michael Davies -  mld9s
* Sam Parsons - sp8hp

In STAT 6021 members of our cohort looked at Transportation Network Company data sets to see if there was a potential relationship between tipping and other indicators, specifically with “transportation network providers” i.e. rideshares such as Uber, Lyft, etc.  At that point in our Data Science journey we did not have the skills or equipment to investigate this question in depth.  

Utilizing machine learning skills from SYS 6018 and applying Spark to this dataset we hope to come up with a more robust set of answers and potentially a better predictor of tipping. With other classification algorithms such as random forest and the heavy-weight data processing of Spark, will we be able to create a more robust predictive model?


Potential Questions from the TNC Data:

* Can it be predicted what fares are most likely to tip the driver?
* Is there a relationship between time of the fare and tipping? (workday stat, bar close, weekday, weekend, etc)
* Is there a relationship between start or end location of the ride and tipping? (downtown pickup, north shore, airport, etc)
* Is there a relationship between length or cost of ride and tipping? (do longer rides result in tips)
* Using this data would we be able to make recommendations to drivers to maximize likelihood of receiving a tip?
* Is the likelihood of tipping changing over time?  Are more rides being tipped?
* Are there re-identification abilities in this dataset? For instance, can we find records for a person who reliably takes a rideshare to/from work every day thereby linking a home address to a work address?




Additionally, joining in additional datasets may yield answers to questions about external factors such as:
* How did news reporting/social media on rideshare companies correlate with tipping?
* What relationship(s) does trip demand have with the stocks of these companies?

Data Source:
The best data source for this appears to be from the City of Chicago, as it is large (169M records and 21 columns), relatively clean, anonymized, and accessible via API.

City of Chicago:
https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p

So far we have only pulled the data down via a CSV.

Code Rubric

* Data Import and PreProcessing | 2 pts

* Data splitting/sampling | 1 pt

* EDA (min two graphs) | 2 pts

* Model construction (min 3 models) | 3 pts

* Model evaluation | 2 pts

In [1]:
# import context manager: SparkSession
from pyspark.sql import SparkSession

# import data types
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType
# import pyspark.sql.types as typ
# import pyspark.sql.functions as F
# import os



from pyspark.sql.types import *

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("mllib_classifier") \
        .config("spark.executor.memory", '21g') \
        .config('spark.executor.cores', '2') \
        .config('spark.executor.instances', '3') \
        .config("spark.driver.memory",'1g') \
        .getOrCreate()
sc = spark.sparkContext

# import data manipulation methods
# from pyspark.ml.feature import Binarizer
from pyspark.ml import Pipeline  
# from pyspark.ml.feature import *  
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import VectorAssembler 
from pyspark.mllib.linalg import Vectors

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import OneHotEncoder
#from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import time

import numpy as np

# Read in our Dataset

## Create a Custom Schema.  
This schema was been primarly determined by using a much smaller dataset and letting spark infer the schema.  We encountered an issue with spark reading in the ENTIRE dataset as NULL when there was a type mismatch.  Only the data we are likely to use later has been assigned to a specific type, otherwise it is left as a string type.

In [2]:
p_df = spark.read.parquet("/../../project/ds5559/Alice_Ed_Michael_Sam_project/final_dataset.parquet")

In [3]:
p_df.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Seconds: integer (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Census_Tract: string (nullable = true)
 |-- Dropoff_Census_Tract: string (nullable = true)
 |-- Pickup_Community_Area: integer (nullable = true)
 |-- Dropoff_Community_Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges_str: double (nullable = true)
 |-- Trip_Total: double (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: integer (nullable = true)
 |-- Pickup_Centroid_Latitude: string (nullable = true)
 |-- Pickup_Centroid_Longitude: string (nullable = true)
 |-- Pickup_Centroid_Location: string (nullable = true)
 |-- Dropoff_Centroid_Latitude: string (nullable = true)
 |-- Dropoff_Centroid_Longitude: string (nullable = true)
 |-- Dropoff_Centroid_Location: string (nullable = true)
 |-- Trip_Start_Timestamp: timestamp (nullable = 

In [ ]:
# create a custom schema.  

customSchema = StructType([
    StructField('Trip_ID', StringType(), True),        
    StructField('Trip_Start_Timestamp', StringType(), True),
    StructField('Trip_End_Timestamp', StringType(), True),
    StructField('Trip_Seconds', DoubleType(), True),
    StructField('Trip_Miles', DoubleType(), True),
    StructField('Pickup_Census_Tract', StringType(), True),
    StructField('Dropoff_Census_Tract', StringType(), True),
    StructField('Pickup_Community_Area', DoubleType(), True),
    StructField('Dropoff_Community_Area', DoubleType(), True),
    StructField("Fare", DoubleType(), True),
    StructField("Tip", DoubleType(), True),
    StructField("Additional_Charges", DoubleType(), True),
    StructField("Trip_Total", StringType(), True),
    StructField("Shared_Trip_Authorized", BooleanType(), True),
    StructField("Trips_Pooled", DoubleType(), True),
    StructField('Pickup_Centroid_Latitude', StringType(), True),
    StructField('Pickup_Centroid_Longitude', StringType(), True),
    StructField('Pickup_Centroid_Location', StringType(), True),
    StructField('Dropoff_Centroid_Latitude', StringType(), True),
    StructField('Dropoff_Centroid_Longitude', StringType(), True),
    StructField('Dropoff_Centroid_Location', StringType(), True)])

#old readin.  Infer is slow for large dataset
#df = spark.read.csv('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv', header = True, inferSchema=True)

#read in the data to a dataframe
df = spark.read.csv('/../../project/ds5559/Alice_Ed_Michael_Sam_project/BigTrips.csv', header = True, schema=customSchema)
df.show(5)

In [ ]:
#Doesn't update if you don't resave the variable

df = df.drop('Pickup_Census_Tract',
             'Dropoff_Census_Tract',
             'Pickup_Centroid_Latitude',
             'Pickup_Centroid_Longitude', 
             'Pickup_Centroid_Location', 
             'Dropoff_Centroid_Latitude', 
             'Dropoff_Centroid_Longitude', 
             'Dropoff_Centroid_Location')

#'Trip_End_Timestamp' keep for now

In [4]:
p_df = p_df.drop('Pickup_Census_Tract',
             'Dropoff_Census_Tract',
             'Pickup_Centroid_Latitude',
             'Pickup_Centroid_Longitude', 
             'Pickup_Centroid_Location', 
             'Dropoff_Centroid_Latitude', 
             'Dropoff_Centroid_Longitude', 
             'Dropoff_Centroid_Location',
             'Day_Month_str')

In [5]:
p_df.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Seconds: integer (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: integer (nullable = true)
 |-- Dropoff_Community_Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges_str: double (nullable = true)
 |-- Trip_Total: double (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: integer (nullable = true)
 |-- Trip_Start_Timestamp: timestamp (nullable = true)
 |-- Trip_End_Timestamp: timestamp (nullable = true)
 |-- PostShutdownFlag: integer (nullable = true)
 |-- Day_Month_str: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- Trip_Year: integer (nullable = true)
 |-- Trip_Month: integer (nullable = true)
 |-- Trip_WeekNumber: integer (nullable = true)
 |-- Trip_DayofWeek: integer (nullable = true)
 |-- Trip_Start_Hour: integer (nullable = true)
 |-- Trip_Start_Minute: i

In [ ]:
df.printSchema()

In [ ]:
df2 = df.sample(False, .05, seed = 2021) #decreased our sample size

In [ ]:
df2.count()

In [ ]:
#delete the big df for now
del (df)

#hopefully that will make things faster 

In [ ]:
p_df.groupby("Pickup_Community_Area").count().show()

In [6]:
#fill our NA community areas

p_df = p_df.na.fill(value=78,subset=['Pickup_Community_Area', 'Dropoff_Community_Area'])

In [ ]:
p_df.groupby("Pickup_Community_Area").count().show()

In [ ]:
#fill our NA community areas

df2 = df2.na.fill(value=78,subset=['Pickup_Community_Area', 'Dropoff_Community_Area'])

In [ ]:
# make a binary tip/no tip indicator
# https://spark.apache.org/docs/2.2.0/ml-features.html#binarizer

#binarized tip seems to be causing problems.  Change its name to label as that is that the packages are expecting

binarizer = Binarizer(threshold=0, inputCol="Tip", outputCol="label")
df2 = binarizer.transform(df2)

In [ ]:
df2.printSchema()

In [ ]:
df2 = df2.withColumn("Trip_Start_TS", F.to_timestamp(F.col("Trip_Start_Timestamp"), "MM/dd/yyyy hh:mm:ss a"))

In [ ]:
df2.printSchema()

In [ ]:
df2 = df2.withColumn('Trip_Year',F.year(F.to_timestamp('Trip_Start_TS'))) \
         .withColumn('Trip_Month',F.month(F.to_timestamp('Trip_Start_TS'))) \
         .withColumn('Trip_WeekNumber',F.weekofyear(F.to_timestamp('Trip_Start_TS'))) \
         .withColumn('Trip_DayofWeek', F.dayofweek(F.col('Trip_Start_TS'))) \
         .withColumn('Trip_Start_Hour', F.hour(F.col('Trip_Start_TS'))) \
         .withColumn('Trip_Start_Minute', F.minute(F.col('Trip_Start_TS'))) \
         .withColumn('Date', F.to_date(F.col('Trip_Start_TS')))
         
df2.show(5, False)

In [ ]:
df2.printSchema()

In [ ]:
# split the data

# our model didn't work on the standard test train split.  Prof. Tashman recomended upscalling the help with the imbalanced dataset.
#from https://spark.apache.org/docs/2.1.0/ml-tuning.html#train-validation-split

train_inital, test = df2.randomSplit([0.8, 0.2], seed=2021)

# cahce our test values for later speed
test.cache()

# oversampleing code sample
# https://stackoverflow.com/questions/53273133/how-to-perform-up-sampling-using-sample-functionpy-spark

df_a = train_inital.filter(train_inital['label'] == 0)
df_b = train_inital.filter(train_inital['label'] == 1)

org_a_count = df_a.count()
org_b_count = df_b.count() 


ratio = df_a.count() / df_b.count()
# print(ratio)

df_b_overampled = df_b.sample(withReplacement=True, fraction=ratio, seed=2021)

# cahce our train values for later speed
train = df_a.unionAll(df_b_overampled).cache()

df_af = train.filter(train_inital['label'] == 0)
df_bf = train.filter(train_inital['label'] == 1)
fin_a_count = df_af.count()
fin_b_count = df_bf.count() 

print("Original No Tip Count: ", org_a_count)
print("Original Tip Count   : ", org_b_count)
print("")
print("Final No Tip Count   : ", fin_a_count)
print("Final Tip Count      : ", fin_b_count)


In [10]:
# split the data

# our model didn't work on the standard test train split.  Prof. Tashman recomended upscalling the help with the imbalanced dataset.
#from https://spark.apache.org/docs/2.1.0/ml-tuning.html#train-validation-split

train, test = p_df.randomSplit([0.8, 0.2], seed=2021)

# cahce our test values for later speed
#test.cache()

# oversampleing code sample
# https://stackoverflow.com/questions/53273133/how-to-perform-up-sampling-using-sample-functionpy-spark

org_a_count = train.filter(train['label'] == 0).count()
org_b_count = train.filter(train['label'] == 1).count()

print("Original No Tip Count: ", org_a_count)
print("Original Tip Count   : ", org_b_count)

Original No Tip Count:  1900289
Original Tip Count   :  1900435


In [40]:
test.cache()
train.cache()

DataFrame[Trip_ID: string, Trip_Seconds: int, Trip_Miles: double, Pickup_Community_Area: int, Dropoff_Community_Area: int, Fare: double, Tip: double, Additional_Charges_str: double, Trip_Total: double, Shared_Trip_Authorized: boolean, Trips_Pooled: int, Trip_Start_Timestamp: timestamp, Trip_End_Timestamp: timestamp, PostShutdownFlag: int, Day_Month_str: string, label: int, Trip_Year: int, Trip_Month: int, Trip_WeekNumber: int, Trip_DayofWeek: int, Trip_Start_Hour: int, Trip_Start_Minute: int, Trip_End_Hour: int, Date: date]

In [11]:
#pre processing handled the imbalanced dataset.  No further balancing is required.  Cache our datasets for speed.

test.cache()
# train = train_inital
train.cache()
del (p_df)

In [ ]:
#just as a reminder what was the truth in our test data?

dft_a = test.filter(train_inital['label'] == 0)
dft_b = test.filter(train_inital['label'] == 1)
count_test_a = dft_a.count()
count_test_b = dft_b.count()
print(count_test_a)
print(count_test_b)


In [ ]:
del (df2)

### Evaluation Functions (UDF)

In [44]:
def cmacc(pred):
    t0 = time.time()
    # make a confusion matrix and return the accuracy
    # select predictions and labels from prediction transform as rdd as there isn't a DF function for this
    pred_rdd= pred.select('prediction').rdd.flatMap(lambda x: x)
    label_rdd = pred.select('label').rdd.flatMap(lambda x: x).map(lambda x: float(x))
    
    #zip them together
    predictionAndLabels =  pred_rdd.zip(label_rdd)
    
    #metrics transform
    metrics = MulticlassMetrics(predictionAndLabels)
    
    metrics2 = BinaryClassificationMetrics(predictionAndLabels)
    
    #make our confusion matrix
    cm = metrics.confusionMatrix().toArray()

    #calc accuracy from confusion matrix
    
    acc = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
    
    # McM accuracy
    
    acc2 = metrics.accuracy
    
    #calc area under curve
    auc = metrics2.areaUnderROC
    
    prc = metrics2.areaUnderPR
    
    print("Confusion Matrix")
    print(cm)
    print()
    print("Accuracy from Confusion Matrix: ", acc)
    print()
    print("Accuracy from MulticlassMetrics: ", acc2)
    print()     
    print("Area Under the ROC", auc)
    print()
    print("Area Under the PR Curve", prc)
    print('-'*50)
    print("Metrics2 time:", time.time() - t0)

In [45]:
def cmacc2(pred):
    # make a confusion matrix and return the accuracy
    # select predictions and labels from prediction transform as rdd as there isn't a DF function for this
    
    t0 = time.time()
    
    pred_rdd= pred.select('prediction').rdd.flatMap(lambda x: x)
    label_rdd = pred.select('label').rdd.flatMap(lambda x: x).map(lambda x: float(x))
    
    #zip them together
    predictionAndLabels =  pred_rdd.zip(label_rdd)
    
    #metrics transform
    metrics = MulticlassMetrics(predictionAndLabels)
    
    metrics2 = BinaryClassificationMetrics(predictionAndLabels)
    
    #make our confusion matrix
    cm = metrics.confusionMatrix().toArray()

    #calc accuracy from confusion matrix
    
    acc = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
    
    #McM accuracy
    acc2 = metrics.accuracy
    
    #calc area under curve
    auc = metrics2.areaUnderROC
    
    prc = metrics2.areaUnderPR
    
    precision = metrics.precision()
    recall = metrics.recall()
    f1Score = metrics.fMeasure()
       
    print("Confusion Matrix")
    print(cm)
    print()
    print("Accuracy from Confusion Matrix: ", acc)
    print("Accuracy from MulticlassMetrics: ", acc2)
    print()
    print("Area Under the ROC", auc)
    print()
    print("Area Under the PR Curve", prc)
   
    print("Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)
    
    # Weighted stats
    print("Weighted recall = %s" % metrics.weightedRecall)
    print("Weighted precision = %s" % metrics.weightedPrecision)
    print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
    print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
    print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)
    print('-'*50)
    print("Metrics2 time:", time.time() - t0)

# Pipeline for LR

In [14]:
train.printSchema()

root
 |-- Trip_ID: string (nullable = true)
 |-- Trip_Seconds: integer (nullable = true)
 |-- Trip_Miles: double (nullable = true)
 |-- Pickup_Community_Area: integer (nullable = true)
 |-- Dropoff_Community_Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tip: double (nullable = true)
 |-- Additional_Charges_str: double (nullable = true)
 |-- Trip_Total: double (nullable = true)
 |-- Shared_Trip_Authorized: boolean (nullable = true)
 |-- Trips_Pooled: integer (nullable = true)
 |-- Trip_Start_Timestamp: timestamp (nullable = true)
 |-- Trip_End_Timestamp: timestamp (nullable = true)
 |-- PostShutdownFlag: integer (nullable = true)
 |-- Day_Month_str: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- Trip_Year: integer (nullable = true)
 |-- Trip_Month: integer (nullable = true)
 |-- Trip_WeekNumber: integer (nullable = true)
 |-- Trip_DayofWeek: integer (nullable = true)
 |-- Trip_Start_Hour: integer (nullable = true)
 |-- Trip_Start_Minute: i

In [51]:
# One Hot Encoding for models (Michael sugested that we didn't need it in each model step)

#onehotencoder to pickup
ohe_pu = OneHotEncoder(inputCol="Pickup_Community_Area", outputCol="Pickup_Community_Area_vec")

#onehotencoder to dropoff
ohe_do = OneHotEncoder(inputCol="Dropoff_Community_Area", outputCol="Dropoff_Community_Area_vec")

#onehotencoder to weekNumber
ohe_twn = OneHotEncoder(inputCol="Trip_WeekNumber", outputCol="Trip_WeekNumber_vec")

#onehotencoder to dayOfWeek
ohe_dw = OneHotEncoder(inputCol="Trip_DayofWeek", outputCol="Trip_DayofWeek_vec")

#onehotencoder to startHour
ohe_sh = OneHotEncoder(inputCol="Trip_Start_Hour", outputCol="Trip_Start_Hour_vec")

#onehotencoder to startMinute
ohe_sm = OneHotEncoder(inputCol="Trip_Start_Minute", outputCol="Trip_Start_Minute_vec")

### Baseline Pipeline Logistic Regression Model

In [52]:
# our colulms for lr
predictor_col_for_lr = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges_str',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # ,'Date' not supported datatype

#assemble the vector for lr
lr_va = VectorAssembler(inputCols=predictor_col_for_lr, outputCol="features") 

#scale our lr
lr_scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

#what do we want to do if we are doing a parameter search? make the parameters as variables and just do a loop?
#we learned that this week.  May also need to add in cv step

lr = LogisticRegression(maxIter=10,
                        regParam=0.1, #org 0.1
                        elasticNetParam=0.3, #org 0.3
                        featuresCol="features",
                        labelCol="label")

# Build the pipeline
lr_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, lr_va, lr_scaler, lr])

#time check
t0 = time.time()

# Fit the pipeline
lr_model = lr_pipeline.fit(train)

# Make a prediction
lr_prediction = lr_model.transform(test)

print("train/test time:", time.time() - t0)

train/test time: 27.292113780975342


In [53]:
#time check
cmacc(lr_prediction)


Confusion Matrix
[[314316. 161081.]
 [228670. 246078.]]

Accuracy from Confusion Matrix:  0.5897983991917023

Accuracy from MulticlassMetrics:  0.5897983991917023

Area Under the ROC 0.5897496184507037

Area Under the PR Curve 0.579158186860823
--------------------------------------------------
Metrics2 time: 8.698458433151245


In [54]:
#time check
cmacc2(lr_prediction)

Confusion Matrix
[[314316. 161081.]
 [228670. 246078.]]

Accuracy from Confusion Matrix:  0.5897983991917023
Accuracy from MulticlassMetrics:  0.5897983991917023

Area Under the ROC 0.5897496184507037

Area Under the PR Curve 0.579158186860823
Summary Stats
Precision = 0.5897983991917023
Recall = 0.5897983991917023
F1 Score = 0.5897983991917023
Weighted recall = 0.5897983991917023
Weighted precision = 0.5916132346574163
Weighted F(1) Score = 0.5876918784941367
Weighted F(0.5) Score = 0.5893040171743007
Weighted false positive rate = 0.41029916229029484
--------------------------------------------------
Metrics2 time: 10.951287984848022


The above confirms the results of the first pass of the grid search from tuning below

### Baseline Pipeline for Random Forrest

In [55]:
# our colulms for rf
predictor_col_for_rf = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges_str',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

# assemble feature vector
rf_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 

# set classifier
rf = RandomForestClassifier(labelCol="label", 
                            featuresCol="features", 
                            numTrees=10)

# Build the pipeline
rf_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, rf_va, rf])

#time check
t0 = time.time()

# Fit the pipeline
rf_model = rf_pipeline.fit(train)

# Make a prediction
rf_prediction = rf_model.transform(test)

print("Train/Test time:", time.time() - t0)

# Select (prediction, true label) and compute test error
rf_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
#metric options f1|accuracy|weightedPrecision|weightedRecall

rf_accuracy = rf_evaluator.evaluate(rf_prediction)

print("Test Error = %g" % (1.0 - rf_accuracy))
print("Accuracy: " , rf_accuracy)

rfModel2 = rf_model.stages[7]
print(rfModel2)  # summary only

Train/Test time: 31.855722188949585
Test Error = 0.427661
Accuracy:  0.5723389587905005
RandomForestClassificationModel (uid=RandomForestClassifier_96b610c6dba2) with 10 trees


In [56]:
cmacc2(rf_prediction)

Confusion Matrix
[[381650.  93747.]
 [312593. 162155.]]

Accuracy from Confusion Matrix:  0.5723389587905005
Accuracy from MulticlassMetrics:  0.5723389587905005

Area Under the ROC 0.5721814320872429

Area Under the PR Curve 0.5895443911731173
Summary Stats
Precision = 0.5723389587905005
Recall = 0.5723389587905005
F1 Score = 0.5723389587905005
Weighted recall = 0.5723389587905003
Weighted precision = 0.5916693419314419
Weighted F(1) Score = 0.5483007926209974
Weighted F(0.5) Score = 0.5639335246559836
Weighted false positive rate = 0.42797609461601455
--------------------------------------------------
Metrics2 time: 11.051759719848633


### Baseline Pipeline for Gradient Boosted Trees

In [57]:
# our colulms for gbt
predictor_col_for_gbt = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges_str',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

gbt_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 

# set classifier
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=5)

# Chain indexers and GBT in a Pipeline
gbt_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, gbt_va, gbt]) #labelIndexer, featureIndexer

#time check
t0 = time.time()

# Train model.
gbt_model = gbt_pipeline.fit(train)

# Make predictions.
gbt_prediction = gbt_model.transform(test)

print("Train/Test time:", time.time() - t0)

# Select (prediction, true label) and compute test error
gbt_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
gbt_accuracy = gbt_evaluator.evaluate(gbt_prediction)
print("Test Error = %g" % (1.0 - gbt_accuracy))
print('gbt accuracy = ', gbt_accuracy)
gbtModel = gbt_model.stages[7]
print(gbtModel)  # summary only

Train/Test time: 123.6289427280426
Test Error = 0.405872
gbt accuracy =  0.5941282646332928
GBTClassificationModel (uid=GBTClassifier_ea139a21856c) with 5 trees


In [58]:
cmacc2(gbt_prediction)

Confusion Matrix
[[329258. 146139.]
 [239498. 235250.]]

Accuracy from Confusion Matrix:  0.5941282646332928
Accuracy from MulticlassMetrics:  0.5941282646332928

Area Under the ROC 0.5940609600027235

Area Under the PR Curve 0.5872707403342221
Summary Stats
Precision = 0.5941282646332928
Recall = 0.5941282646332928
F1 Score = 0.5941282646332928
Weighted recall = 0.5941282646332928
Weighted precision = 0.5978537258908767
Weighted F(1) Score = 0.590143502463202
Weighted F(0.5) Score = 0.5933010409575987
Weighted false positive rate = 0.40600634462784574
--------------------------------------------------
Metrics2 time: 10.66636872291565


### Pipeline LR with Tuning


In [59]:
# our colulms for lr
predictor_col_for_lr = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges_str',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

#assemble the vector or LR

lr_va = VectorAssembler(inputCols=predictor_col_for_lr, outputCol="features") 

#scale our LR

lr_scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

#what do we want to do if we are doing a parameter search? make the parameters as variables and just do a loop?
#we learned that this week.  May also need to add in cv step

lr = LogisticRegression(featuresCol="features",
                        labelCol="label") # regParam=0.1, elasticNetParam=0.3, maxIter=10,

# Build the pipeline
lr_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, lr_va, lr_scaler, lr])

# Set up the parameter grid
lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.02, 0.03]) \
    .addGrid(lr.elasticNetParam, [0.1, 0.15]) \
    .addGrid(lr.maxIter, [5, 10]) \
    .build()

print('len(lr_paramGrid): {}'.format(len(lr_paramGrid)))
'''
25k set
best from tuning inital (accuracy):
addGrid(lr.regParam, [0.03, 0.05, 0.07]) \
.addGrid(lr.elasticNetParam, [0.15, 0.2, 0.25]) \
.addGrid(lr.maxIter, [8, 9, 10, 11, 12]) 

regParam= 0.03
elasticNetParam = 0.15
maxIter = 12
 
Confusion Matrix
[[1842. 2203.]
 [ 193.  595.]]

Accuracy from Confusion Matrix:  0.5042416718394372
Accuracy from MulticlassMetrics:  0.5042416718394372

Area Under the ROC 0.6052265753923187

Area Under the PR Curve 0.2065770273222743
Summary Stats
Precision = 0.5042416718394372
Recall = 0.5042416718394372
F1 Score = 0.5042416718394372
Weighted recall = 0.5042416718394371
Weighted precision = 0.7922492654683645
Weighted F(1) Score = 0.5612342974368173
Weighted F(0.5) Score = 0.6730989071456968
Weighted false positive rate = 0.2937885210547999

2nd round:

lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.02, 0.03]) \
    .addGrid(lr.elasticNetParam, [0.05, 0.1, 0.15]) \
    .addGrid(lr.maxIter, [11, 12, 13, 15]) \

regParam= 0.03
elasticNetParam = 0.15
maxIter = 11

Confusion Matrix
[[1815. 2230.]
 [ 183.  605.]]

Accuracy from Confusion Matrix:  0.5007241878750258
Accuracy from MulticlassMetrics:  0.5007241878750258

Area Under the ROC 0.6082342994108162

Area Under the PR Curve 0.2075564549699384
Summary Stats
Precision = 0.5007241878750258
Recall = 0.5007241878750258
F1 Score = 0.5007241878750258
Weighted recall = 0.5007241878750258
Weighted precision = 0.7950908896146499
Weighted F(1) Score = 0.5572078454446675
Weighted F(0.5) Score = 0.6716684076809212
Weighted false positive rate = 0.28425558905339354

'''

# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
lr_crossval = CrossValidator(estimator=lr_pipeline,
                          estimatorParamMaps=lr_paramGrid,
                          #evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'), #we can pass in our own function if necessary
                          evaluator= MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5)

# you can do a custom evaluator, but it seems to be a lot of work.  https://stackoverflow.com/questions/51404344/custom-evaluator-in-pyspark
# we can use either areaUnderROC or areaUnderPR as defaults for binary.
# f1|accuracy|weightedPrecision|weightedRecall for multiclass

# Run cross-validation, and choose the best set of parameters. Print the training time.

t0 = time.time()
lr_cvModel = lr_crossval.setParallelism(6).fit(train) # train 6 models in parallel

print("train time:", time.time() - t0)


len(lr_paramGrid): 8
train time: 1099.297649383545


In [ ]:
#how to find all our items we can call
#dir(crossval.evaluator)

In [ ]:
#not sure what this metric is... apparently it is RMSE https://projector-video-pdf-converter.datacamp.com/14989/chapter4.pdf
#lr_cvModel.avgMetrics

In [ ]:
# magic code from https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark

# lr_cvModel.getEstimatorParamMaps()[ np.argmax(lr_cvModel.avgMetrics) ]

In [60]:
#https://dsharpc.github.io/SparkMLFlights/
#best?
#cvModel.getEstimatorParamMaps()[ np.argmin(cvModel.avgMetrics) ]

lr_cvModel.getEstimatorParamMaps()[ np.argmin(lr_cvModel.avgMetrics) ]

{Param(parent='LogisticRegression_7d4d6d3af10c', name='regParam', doc='regularization parameter (>= 0).'): 0.03,
 Param(parent='LogisticRegression_7d4d6d3af10c', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.15,
 Param(parent='LogisticRegression_7d4d6d3af10c', name='maxIter', doc='max number of iterations (>= 0).'): 5}

In [61]:
lr_cvModel.getEstimatorParamMaps()[ np.argmax(lr_cvModel.avgMetrics) ]

{Param(parent='LogisticRegression_7d4d6d3af10c', name='regParam', doc='regularization parameter (>= 0).'): 0.02,
 Param(parent='LogisticRegression_7d4d6d3af10c', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.1,
 Param(parent='LogisticRegression_7d4d6d3af10c', name='maxIter', doc='max number of iterations (>= 0).'): 10}

In [62]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
lr_prediction = lr_cvModel.transform(test)

In [63]:
#prediction.show(10)

In [64]:
#cmacc(lr_prediction)

In [65]:
cmacc2(lr_prediction)

Confusion Matrix
[[258980. 216417.]
 [143151. 331597.]]

Accuracy from Confusion Matrix:  0.6215651295328608
Accuracy from MulticlassMetrics:  0.6215651295328608

Area Under the ROC 0.6216176234893992

Area Under the PR Curve 0.5891933635195167
Summary Stats
Precision = 0.6215651295328608
Recall = 0.6215651295328608
F1 Score = 0.6215651295328608
Weighted recall = 0.6215651295328608
Weighted precision = 0.6245670682712414
Weighted F(1) Score = 0.6193217829971516
Weighted F(0.5) Score = 0.6215444248056616
Weighted false positive rate = 0.3783298825540624
--------------------------------------------------
Metrics2 time: 11.47052264213562


# Pipeline RF with Tuning


In [67]:
# our colulms for rf
predictor_col_for_rf = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges_str',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

# assemble feature vector
rf_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 
               
# set classifier
rf = RandomForestClassifier(labelCol="label", 
                            featuresCol="features")
    
# Build the pipeline
rf_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, rf_va, rf])

# Set up the parameter grid
rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 10]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .addGrid(rf.impurity, ["gini", "entropy"])\
    .build()
    #.addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt'])\
    
print('len(rf_paramGrid): {}'.format(len(rf_paramGrid)))

#https://medium.com/rahasak/random-forest-classifier-with-apache-spark-c63b4a23a7cc
#maxDepth, maxBins, impurity, auto and seed 
#.addGrid(randomForestClassifier.impurity, Array("entropy", "gini"))
#name='featureSubsetStrategy', auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]

'''
best from tuning inital (accuracy):
rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 10]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .addGrid(rf.impurity, ["entropy", "gini"])\
    .build()

numTrees = 10
maxDepth = 10
impurity = gini
 
Confusion Matrix
[[2901. 1144.]
 [ 456.  332.]]

Accuracy from Confusion Matrix:  0.6689426857024623
Accuracy from MulticlassMetrics:  0.6689426857024623

Area Under the ROC 0.5692507513819781

Area Under the PR Curve 0.2070259967551608
Summary Stats
Precision = 0.6689426857024623
Recall = 0.6689426857024623
F1 Score = 0.6689426857024623
Weighted recall = 0.6689426857024622
Weighted precision = 0.7599403563099746
Weighted F(1) Score = 0.7038591473392332
Weighted F(0.5) Score = 0.735232181263078
Weighted false positive rate = 0.530441182938506

2nd attempt:

rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 10, 15]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .addGrid(rf.impurity, ["entropy", "gini"])\
    .addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt'])\
    .build()


numTrees = 10
maxDepth = 15
impurity = gini
featureSubsetStrategy = auto

Confusion Matrix
[[2663. 1382.]
 [ 388.  400.]]

Accuracy from Confusion Matrix:  0.6337678460583489
Accuracy from MulticlassMetrics:  0.6337678460583489

Area Under the ROC 0.5829789236570811

Area Under the PR Curve 0.209345437091233
Summary Stats
Precision = 0.6337678460583489
Recall = 0.6337678460583489
F1 Score = 0.6337678460583489
Weighted recall = 0.6337678460583488
Weighted precision = 0.7671159775546591
Weighted F(1) Score = 0.6789410276493224
Weighted F(0.5) Score = 0.7270236282319229
Weighted false positive rate = 0.46780999874418644

'''

# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
rf_crossval = CrossValidator(estimator=rf_pipeline,
                          estimatorParamMaps=rf_paramGrid,
                          #evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'), #we can pass in our own function if necessary
                          evaluator= MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5)

# you can do a custom evaluator, but it seems to be a lot of work.  https://stackoverflow.com/questions/51404344/custom-evaluator-in-pyspark
# we can use either areaUnderROC or areaUnderPR as defaults for binary.
# f1|accuracy|weightedPrecision|weightedRecall for multiclass

# Run cross-validation, and choose the best set of parameters. Print the training time.

t0 = time.time()
cvModel_rf = rf_crossval.setParallelism(6).fit(train) # train 6 models in parallel

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction_rf = cvModel_rf.transform(test)

print("train time:", time.time() - t0)

len(rf_paramGrid): 8
train time: 1360.484427690506


In [69]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction_rf = cvModel_rf.transform(test)

In [70]:
cmacc2(prediction_rf)

Confusion Matrix
[[356675. 118722.]
 [269179. 205569.]]

Accuracy from Confusion Matrix:  0.5917454704281978
Accuracy from MulticlassMetrics:  0.5917454704281978

Area Under the ROC 0.5916371172511604

Area Under the PR Curve 0.5958450337450366
Summary Stats
Precision = 0.5917454704281978
Recall = 0.5917454704281978
F1 Score = 0.5917454704281978
Weighted recall = 0.5917454704281978
Weighted precision = 0.6018802269396942
Weighted F(1) Score = 0.5811974532996568
Weighted F(0.5) Score = 0.5893879719422019
Weighted false positive rate = 0.4084712359258768
--------------------------------------------------
Metrics2 time: 13.019006490707397


In [ ]:
#how to find all our items we can call
#dir(crossval.evaluator)

In [71]:
#not sure what this metric is... rmse
cvModel_rf.avgMetrics

[0.5855043851048654,
 0.5880303476189277,
 0.593714324922341,
 0.5905547727116429,
 0.5811094961300449,
 0.583791821161717,
 0.5952770626796756,
 0.5984883305576575]

In [72]:
# magic code from https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark

cvModel_rf.getEstimatorParamMaps()[ np.argmax(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_0080c2e2c57d', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
 Param(parent='RandomForestClassifier_0080c2e2c57d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='RandomForestClassifier_0080c2e2c57d', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'entropy'}

In [73]:
#https://dsharpc.github.io/SparkMLFlights/
#best?
#cvModel.getEstimatorParamMaps()[ np.argmin(cvModel.avgMetrics) ]

cvModel_rf.getEstimatorParamMaps()[ np.argmin(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_0080c2e2c57d', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
 Param(parent='RandomForestClassifier_0080c2e2c57d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestClassifier_0080c2e2c57d', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}

In [74]:
cvModel_rf.getEstimatorParamMaps()[ np.argmax(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_0080c2e2c57d', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
 Param(parent='RandomForestClassifier_0080c2e2c57d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='RandomForestClassifier_0080c2e2c57d', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'entropy'}

# Pipeline GBT with Tuning

In [75]:
# our colulms for gbt
predictor_col_for_gbt = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges_str',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec'
                        ] # 'Date' not supported datatype

gbt_va = VectorAssembler(inputCols=predictor_col_for_gbt, outputCol="features") 

# Train a GBT model.
gbt = GBTClassifier(labelCol="label", featuresCol="features") #, maxIter=5

# Chain indexers and GBT in a Pipeline
gbt_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, gbt_va, gbt]) #labelIndexer, featureIndexer

# Set up the parameter grid
gbt_paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [5, 10])\
    .addGrid(gbt.maxDepth, [5, 10])\
    .build()

print('len(gbt_paramGrid): {}'.format(len(gbt_paramGrid)))


'''
best from tuning inital (accuracy):
gbt_paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [5, 10, 20]) \
    .build()

maxIter = 20

Confusion Matrix
[[2798. 1247.]
 [ 423.  365.]]

Accuracy from Confusion Matrix:  0.654458928201945
Accuracy from MulticlassMetrics:  0.654458928201945

Area Under the ROC 0.5774580700620556

Area Under the PR Curve 0.2094152550126291
Summary Stats
Precision = 0.654458928201945
Recall = 0.654458928201945
F1 Score = 0.654458928201945
Weighted recall = 0.654458928201945
Weighted precision = 0.7639586098089827
Weighted F(1) Score = 0.6941837869282138
Weighted F(0.5) Score = 0.7327747544723259
Weighted false positive rate = 0.4995427880778336


maxIter = 40

Confusion Matrix
[[2571. 1474.]
 [ 382.  406.]]

Accuracy from Confusion Matrix:  0.6159735154148562
Accuracy from MulticlassMetrics:  0.6159735154148562

Area Under the ROC 0.5754139659791808

Area Under the PR Curve 0.20313239804234073
Summary Stats
Precision = 0.6159735154148562
Recall = 0.6159735154148562
F1 Score = 0.6159735154148562
Weighted recall = 0.6159735154148562
Weighted precision = 0.7638968296438198
Weighted F(1) Score = 0.6646010165217533
Weighted F(0.5) Score = 0.7183436330713325
Weighted false positive rate = 0.4651455834564943


gbt_paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [5, 10])\
    .addGrid(gbt.maxDepth, [5, 10])\
    .build()
    
maxIter = 5
maxDepth = 5

Confusion Matrix
[[2863. 1182.]
 [ 469.  319.]]

Accuracy from Confusion Matrix:  0.6583902338092282
Accuracy from MulticlassMetrics:  0.6583902338092282

Area Under the ROC 0.5563048634335804

Area Under the PR Curve 0.19780050930331394
Summary Stats
Precision = 0.6583902338092282
Recall = 0.6583902338092282
F1 Score = 0.6583902338092282
Weighted recall = 0.6583902338092282
Weighted precision = 0.7537989743798752
Weighted F(1) Score = 0.6950856095348379
Weighted F(0.5) Score = 0.7279222226014918
Weighted false positive rate = 0.5457805069420676
'''

# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
gbt_crossval = CrossValidator(estimator=gbt_pipeline,
                          estimatorParamMaps=gbt_paramGrid,
                          #evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'), #we can pass in our own function if necessary
                          evaluator= MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5)

# you can do a custom evaluator, but it seems to be a lot of work.  https://stackoverflow.com/questions/51404344/custom-evaluator-in-pyspark
# we can use either areaUnderROC or areaUnderPR as defaults for binary.
# f1|accuracy|weightedPrecision|weightedRecall for multiclass

# Run cross-validation, and choose the best set of parameters. Print the training time.

t0 = time.time()
cvModel_gbt = gbt_crossval.setParallelism(6).fit(train) # train 6 models in parallel
print("train time:", time.time() - t0)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction_gbt = cvModel_gbt.transform(test)

len(gbt_paramGrid): 4
train time: 4965.340165376663


In [76]:
cmacc2(prediction_gbt)

Confusion Matrix
[[306188. 169209.]
 [200380. 274368.]]

Accuracy from Confusion Matrix:  0.6110183182566872
Accuracy from MulticlassMetrics:  0.6110183182566872

Area Under the ROC 0.6109957281138295

Area Under the PR Curve 0.5934476809925365
Summary Stats
Precision = 0.6110183182566872
Recall = 0.6110183182566872
F1 Score = 0.6110183182566872
Weighted recall = 0.6110183182566871
Weighted precision = 0.6114808613609575
Weighted F(1) Score = 0.6105904720912074
Weighted F(0.5) Score = 0.6109656999099813
Weighted false positive rate = 0.3890268620290281
--------------------------------------------------
Metrics2 time: 13.175491571426392


In [ ]:
#how to find all our items we can call
#dir(crossval.evaluator)

In [77]:
#not sure what this metric is...
cvModel_gbt.avgMetrics

[0.5952295992563201,
 0.6052778306592042,
 0.6014306543937586,
 0.6105873712720494]

In [78]:
# magic code from https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark

#cvModel_gbt.getEstimatorParamMaps()[ np.argmax(cvModel_rf.avgMetrics) ]

cvModel_gbt.getEstimatorParamMaps()[np.argmax(cvModel_gbt.avgMetrics)]

{Param(parent='GBTClassifier_aee1714e173e', name='maxIter', doc='max number of iterations (>= 0).'): 10,
 Param(parent='GBTClassifier_aee1714e173e', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10}

In [79]:
cvModel_gbt.getEstimatorParamMaps()[np.argmin(cvModel_gbt.avgMetrics)]

{Param(parent='GBTClassifier_aee1714e173e', name='maxIter', doc='max number of iterations (>= 0).'): 5,
 Param(parent='GBTClassifier_aee1714e173e', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5}

In [ ]:
#prediction.show(10)

# Pipeline LR with CV and no Tuning


In [81]:
# our colulms for lr
predictor_col_for_lr = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges_str',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec',
                        'PostShutdownFlag'
                        ] # 'Date' not supported datatype

#assemble the vector ror lr
lr_va = VectorAssembler(inputCols=predictor_col_for_lr, outputCol="features") 

#scale our LR

lr_scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

#what do we want to do if we are doing a parameter search? make the parameters as variables and just do a loop?
#we learned that this week.  May also need to add in cv step

lr = LogisticRegression(featuresCol="features",
                        labelCol="label") # regParam=0.1, elasticNetParam=0.3, maxIter=10,

# Build the pipeline
lr_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, lr_va, lr_scaler, lr])

# Set up the parameter grid
lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.03]) \
    .addGrid(lr.elasticNetParam, [0.15]) \
    .addGrid(lr.maxIter, [5]) \
    .build()

print('len(lr_paramGrid): {}'.format(len(lr_paramGrid)))
'''
lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01]) \
    .addGrid(lr.elasticNetParam, [0.3]) \
    .addGrid(lr.maxIter, [10]) \
    .build()

train time: 140.95259499549866
Confusion Matrix
[[205206. 204047.]
 [ 21926.  59887.]]

Accuracy from Confusion Matrix:  0.5398317130487551
Accuracy from MulticlassMetrics:  0.5398317130487551

Area Under the ROC 0.6167072883197446

Area Under the PR Curve 0.2188213722126973
Summary Stats
Precision = 0.5398317130487551
Recall = 0.5398317130487551
F1 Score = 0.5398317130487551
Weighted recall = 0.5398317130487552
Weighted precision = 0.7907482614736813
Weighted F(1) Score = 0.5951821122400512
Weighted F(0.5) Score = 0.6927412010642031
Weighted false positive rate = 0.3064171364092659


lr_paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01]) \
    .addGrid(lr.elasticNetParam, [0.3]) \
    .addGrid(lr.maxIter, [10]) \
    .build()

train time: 103.50798845291138
Confusion Matrix
[[190906. 218347.]
 [ 19609.  62204.]]

Accuracy from Confusion Matrix:  0.5154296978410234
Accuracy from MulticlassMetrics:  0.5154296978410234

Area Under the ROC 0.61339677414919

Area Under the PR Curve 0.21511547038697104
Summary Stats
Precision = 0.5154296978410234
Recall = 0.5154296978410234
F1 Score = 0.5154296978410234
Weighted recall = 0.5154296978410234
Weighted precision = 0.792707390100297
Weighted F(1) Score = 0.5706182258743312
Weighted F(0.5) Score = 0.678770901909123
Weighted false positive rate = 0.2886361495426433

'''

# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
lr_crossval = CrossValidator(estimator=lr_pipeline,
                          estimatorParamMaps=lr_paramGrid,
                          #evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'), #we can pass in our own function if necessary
                          evaluator= MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5)

# you can do a custom evaluator, but it seems to be a lot of work.  https://stackoverflow.com/questions/51404344/custom-evaluator-in-pyspark
# we can use either areaUnderROC or areaUnderPR as defaults for binary.
# f1|accuracy|weightedPrecision|weightedRecall for multiclass

# Run cross-validation, and choose the best set of parameters. Print the training time.

t0 = time.time()
lr_cvModel = lr_crossval.setParallelism(5).fit(train) # train 5 models in parallel
print("train time:", time.time() - t0)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
lr_prediction = lr_cvModel.transform(test)

cmacc2(lr_prediction)

len(lr_paramGrid): 1
train time: 181.59805846214294
Confusion Matrix
[[288388. 187009.]
 [179953. 294795.]]

Accuracy from Confusion Matrix:  0.6137831594125107
Accuracy from MulticlassMetrics:  0.6137831594125107

Area Under the ROC 0.6137880517373892

Area Under the PR Curve 0.5905923489395983
Summary Stats
Precision = 0.6137831594125107
Recall = 0.6137831594125107
F1 Score = 0.6137831594125107
Weighted recall = 0.6137831594125107
Weighted precision = 0.6138121835166755
Weighted F(1) Score = 0.6137638181652783
Weighted F(0.5) Score = 0.6137847105445953
Weighted false positive rate = 0.38620705593773236
--------------------------------------------------
Metrics2 time: 12.068689346313477


# Pipeline RF with CV and no Tuning

In [86]:
# our colulms for rf
predictor_col_for_rf = ['Trip_Seconds',
                        'Trip_Miles',
                        'Fare',
                        'Additional_Charges_str',
                        'Shared_Trip_Authorized',
                        'Trips_Pooled',
                        'Pickup_Community_Area_vec',
                        'Dropoff_Community_Area_vec',
                        'Trip_Year', 
                        'Trip_Month',
                        'Trip_WeekNumber_vec', 
                        'Trip_DayofWeek_vec', 
                        'Trip_Start_Hour_vec',
                        'Trip_Start_Minute_vec',
                        'PostShutdownFlag'
                        ]

# assemble feature vector
rf_va = VectorAssembler(inputCols=predictor_col_for_rf, outputCol="features") 

# set classifier
rf = RandomForestClassifier(labelCol="label", 
                            featuresCol="features")

# # Build the pipeline
# rf_pipeline = Pipeline(stages=[ohe_pu, ohe_do, rf_va, rf])

# Build the pipeline
rf_pipeline = Pipeline(stages=[ohe_pu, ohe_do, ohe_twn, ohe_dw, ohe_sh, ohe_sm, rf_va, rf])

# Set up the parameter grid
rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10]) \
    .addGrid(rf.maxDepth, [5]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()
    #.addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt'])\
    

print('len(rf_paramGrid): {}'.format(len(rf_paramGrid)))

#https://medium.com/rahasak/random-forest-classifier-with-apache-spark-c63b4a23a7cc
#maxDepth, maxBins, impurity, auto and seed 
#.addGrid(randomForestClassifier.impurity, Array("entropy", "gini"))
#name='featureSubsetStrategy', auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]

'''

rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5]) \
    .addGrid(rf.maxDepth, [10]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()

Confusion Matrix
[[310741.  98512.]
 [ 48819.  32994.]]

Accuracy from Confusion Matrix:  0.6999771924751459
Accuracy from MulticlassMetrics:  0.6999771924751459

Area Under the ROC 0.5812869028226872

Area Under the PR Curve 0.22574477490483819
Summary Stats
Precision = 0.6999771924751459
Recall = 0.6999771924751459
F1 Score = 0.6999771924751459
Weighted recall = 0.6999771924751459
Weighted precision = 0.7620428175754068
Weighted F(1) Score = 0.725226449674273
Weighted F(0.5) Score = 0.746087327758354
Weighted false positive rate = 0.5374033868297713

rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [5, 10]) \
    .addGrid(rf.maxDepth, [10]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()
    #.addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt'])\

Confusion Matrix
[[314738.  94515.]
 [ 49387.  32426.]]

Accuracy from Confusion Matrix:  0.7069599605755642
Accuracy from MulticlassMetrics:  0.7069599605755642

Area Under the ROC 0.5826988592158698

Area Under the PR Curve 0.22862746418070495
Summary Stats
Precision = 0.7069599605755642
Recall = 0.7069599605755642
F1 Score = 0.7069599605755642
Weighted recall = 0.7069599605755641
Weighted precision = 0.7629191089280811
Weighted F(1) Score = 0.7300846426136949
Weighted F(0.5) Score = 0.7487527818041555
Weighted false positive rate = 0.5415622421438244

rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()
    
Confusion Matrix
[[314738.  94515.]
 [ 49387.  32426.]]

Accuracy from Confusion Matrix:  0.7069599605755642
Accuracy from MulticlassMetrics:  0.7069599605755642

Area Under the ROC 0.5826988592158698

Area Under the PR Curve 0.22862746418070495
Summary Stats
Precision = 0.7069599605755642
Recall = 0.7069599605755642
F1 Score = 0.7069599605755642
Weighted recall = 0.7069599605755641
Weighted precision = 0.7629191089280811
Weighted F(1) Score = 0.7300846426136949
Weighted F(0.5) Score = 0.7487527818041555
Weighted false positive rate = 0.5415622421438244


rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [15]) \
    .addGrid(rf.maxDepth, [10]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()

Confusion Matrix
[[314741.  94512.]
 [ 49347.  32466.]]

Accuracy from Confusion Matrix:  0.7070475251799148
Accuracy from MulticlassMetrics:  0.7070475251799148

Area Under the ROC 0.5829469843572036

Area Under the PR Curve 0.22881720820050683
Summary Stats
Precision = 0.7070475251799148
Recall = 0.7070475251799148
F1 Score = 0.7070475251799148
Weighted recall = 0.7070475251799148
Weighted precision = 0.7630392635253729
Weighted F(1) Score = 0.7301782260281282
Weighted F(0.5) Score = 0.7488602512258918
Weighted false positive rate = 0.5411535564655076

rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [15]) \
    .addGrid(rf.maxDepth, [15]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()

Confusion Matrix
[[283025. 126228.]
 [ 40908.  40905.]]

Accuracy from Confusion Matrix:  0.6596465648202074
Accuracy from MulticlassMetrics:  0.6596465648202074

Area Under the ROC 0.5957732705112914

Area Under the PR Curve 0.22520890081666428
Summary Stats
Precision = 0.6596465648202074
Recall = 0.6596465648202074
F1 Score = 0.6596465648202074
Weighted recall = 0.6596465648202074
Weighted precision = 0.76892652146907
Weighted F(1) Score = 0.6981671019368805
Weighted F(0.5) Score = 0.7371252023470279
Weighted false positive rate = 0.4681000237976247

rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [15]) \
    .addGrid(rf.maxDepth, [5]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()

Confusion Matrix
[[309640.  99613.]
 [ 49083.  32730.]]

Accuracy from Confusion Matrix:  0.6971975253835533
Accuracy from MulticlassMetrics:  0.6971975253835533

Area Under the ROC 0.5783283336102997

Area Under the PR Curve 0.22310156202954437
Summary Stats
Precision = 0.6971975253835533
Recall = 0.6971975253835533
F1 Score = 0.6971975253835533
Weighted recall = 0.6971975253835534
Weighted precision = 0.7605687622026887
Weighted F(1) Score = 0.7229589255192178
Weighted F(0.5) Score = 0.7442644365033869
Weighted false positive rate = 0.540540858162954

rf_paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [20]) \
    .addGrid(rf.maxDepth, [10]) \
    .addGrid(rf.impurity, ["gini"])\
    .build()

Confusion Matrix
[[287361. 121892.]
 [ 43803.  38010.]]

Accuracy from Confusion Matrix:  0.6625809972590242
Accuracy from MulticlassMetrics:  0.6625809972590242

Area Under the ROC 0.5833779398869698

Area Under the PR Curve 0.21867308547708184
Summary Stats
Precision = 0.6625809972590242
Recall = 0.6625809972590242
F1 Score = 0.6625809972590242
Weighted recall = 0.6625809972590243
Weighted precision = 0.7627667057602532
Weighted F(1) Score = 0.6992915166799253
Weighted F(0.5) Score = 0.7344847606612861
Weighted false positive rate = 0.4958251174850846
'''



len(rf_paramGrid): 1


'\n\nrf_paramGrid = ParamGridBuilder()     .addGrid(rf.numTrees, [5])     .addGrid(rf.maxDepth, [10])     .addGrid(rf.impurity, ["gini"])    .build()\n\nConfusion Matrix\n[[310741.  98512.]\n [ 48819.  32994.]]\n\nAccuracy from Confusion Matrix:  0.6999771924751459\nAccuracy from MulticlassMetrics:  0.6999771924751459\n\nArea Under the ROC 0.5812869028226872\n\nArea Under the PR Curve 0.22574477490483819\nSummary Stats\nPrecision = 0.6999771924751459\nRecall = 0.6999771924751459\nF1 Score = 0.6999771924751459\nWeighted recall = 0.6999771924751459\nWeighted precision = 0.7620428175754068\nWeighted F(1) Score = 0.725226449674273\nWeighted F(0.5) Score = 0.746087327758354\nWeighted false positive rate = 0.5374033868297713\n\nrf_paramGrid = ParamGridBuilder()     .addGrid(rf.numTrees, [5, 10])     .addGrid(rf.maxDepth, [10])     .addGrid(rf.impurity, ["gini"])    .build()\n    #.addGrid(rf.featureSubsetStrategy, [\'auto\', \'sqrt\'])\nConfusion Matrix\n[[314738.  94515.]\n [ 49387.  32426.

In [87]:
# Treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
rf_crossval = CrossValidator(estimator=rf_pipeline,
                          estimatorParamMaps=rf_paramGrid,
                          #evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'), #we can pass in our own function if necessary
                          evaluator= MulticlassClassificationEvaluator(metricName='accuracy'),
                          numFolds=5)

# you can do a custom evaluator, but it seems to be a lot of work.  https://stackoverflow.com/questions/51404344/custom-evaluator-in-pyspark
# we can use either areaUnderROC or areaUnderPR as defaults for binary.
# f1|accuracy|weightedPrecision|weightedRecall for multiclass

In [88]:
#how to find all our items we can call
#dir(crossval.evaluator)

In [89]:
# Run cross-validation, and choose the best set of parameters. Print the training time.

t0 = time.time()
cvModel_rf = rf_crossval.setParallelism(5).fit(train) # train 5 models in parallel
print("train time:", time.time() - t0)

train time: 227.34068822860718


In [90]:
#not sure what this metric is... rmse
cvModel_rf.avgMetrics

[0.5856482067911174]

In [91]:
# magic code from https://stackoverflow.com/questions/36697304/how-to-extract-model-hyper-parameters-from-spark-ml-in-pyspark

cvModel_rf.getEstimatorParamMaps()[ np.argmax(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_239c676a5646', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
 Param(parent='RandomForestClassifier_239c676a5646', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestClassifier_239c676a5646', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}

In [92]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction_rf = cvModel_rf.transform(test)

In [93]:
#prediction.show(10)

In [94]:
cmacc2(prediction_rf)

Confusion Matrix
[[352170. 123227.]
 [267404. 207344.]]

Accuracy from Confusion Matrix:  0.5888722247656937
Accuracy from MulticlassMetrics:  0.5888722247656937

Area Under the ROC 0.588768384905192

Area Under the PR Curve 0.5913022696245284
Summary Stats
Precision = 0.5888722247656937
Recall = 0.5888722247656937
F1 Score = 0.5888722247656937
Weighted recall = 0.5888722247656937
Weighted precision = 0.5977981769216145
Weighted F(1) Score = 0.5791369373921629
Weighted F(0.5) Score = 0.5865359799475878
Weighted false positive rate = 0.4113354549553096
--------------------------------------------------
Metrics2 time: 12.298969507217407


In [95]:
#https://dsharpc.github.io/SparkMLFlights/
#best?
#cvModel.getEstimatorParamMaps()[ np.argmin(cvModel.avgMetrics) ]

cvModel_rf.getEstimatorParamMaps()[ np.argmin(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_239c676a5646', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
 Param(parent='RandomForestClassifier_239c676a5646', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestClassifier_239c676a5646', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}

In [96]:
cvModel_rf.getEstimatorParamMaps()[ np.argmax(cvModel_rf.avgMetrics) ]

{Param(parent='RandomForestClassifier_239c676a5646', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
 Param(parent='RandomForestClassifier_239c676a5646', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestClassifier_239c676a5646', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}